In [1]:
# Mithil Patel
# DSC 540 - Milestone 4 Assignment
# 11/06/2022

# Importing libraries
import requests as r
import pandas as pd
import datetime
import time

# importing coordinates (latitude, longitude) dataframe
cor_df = pd.read_csv("world_country_and_usa_states_latitude_and_longitude_values.csv")

# Dropping rows with NaN values
cor_df.dropna(subset=['latitude'], inplace=True)

# assigned regular string date
start_date = datetime.datetime(2021,1, 1)
end_date = datetime.datetime(2021,1,1)

# Convert to Unix for api parameter
start = int(time.mktime(start_date.timetuple()))
end = int(time.mktime(end_date.timetuple()))

# Making a api call to collect air quality data
api_key = '1237b1ff10b2a1f521c5aac9db19281a'
lat, lon = cor_df.iloc[0, 1], cor_df.iloc[0, 2]
response = r.get(f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={api_key}")

# Creating a pandas dataframe using the data collected from api
data = response.json()
data = data['list'][0]['main']
df = pd.json_normalize(data)
df['country'] = cor_df.iloc[0, 3]
df

,aqi,country
0,1,Andorra


In [2]:
# Sending api request for each coordinates and appending collected data values to the dataframe
for i in range(1,len(cor_df)):
    country = cor_df.iloc[i,3]
    lat, lon = cor_df.iloc[i, 1], cor_df.iloc[i, 2]
    response = r.get(f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={api_key}")
    data = response.json()
    data = data['list'][0]['main']['aqi']
    df.loc[len(df.index)] = [data,country]

# Displaying dataframe
df.head()

,aqi,country
0,1,Andorra
1,5,United Arab Emirates
2,1,Afghanistan
3,2,Antigua and Barbuda
4,2,Anguilla


In [3]:
# Transformation 1: Rearrange columns to display country column first
# Reason: To remain consistent with other dataframe style
api_df = df[['country','aqi']]
api_df.head()

,country,aqi
0,Andorra,1
1,United Arab Emirates,5
2,Afghanistan,1
3,Antigua and Barbuda,2
4,Anguilla,2


In [4]:
# Transformation 2: rename column names
# Reason: To increase readability by properly naming the columns
api_df.rename({'aqi': 'air_quality'}, axis=1, inplace=True)
api_df.head()

,country,air_quality
0,Andorra,1
1,United Arab Emirates,5
2,Afghanistan,1
3,Antigua and Barbuda,2
4,Anguilla,2


In [5]:
# Checking for duplicates
api_df[api_df.duplicated()]

,country,air_quality


In [6]:
# Checking for null values
api_df.isnull().sum()

country        0
air_quality    0
dtype: int64

In [7]:
# Transformation 3: Arranging country column in ascending order
# Reason: Other two dataframe are arranged in ascending order; therefore, arranging will make
#         it easier to concatenate all dataframes

api_df.sort_values('country', ascending=True)

,country,air_quality
2,Afghanistan,1
5,Albania,1
58,Algeria,5
11,American Samoa,1
0,Andorra,1
...,...,...
236,Wallis and Futuna,1
62,Western Sahara,5
239,Yemen,2
242,Zambia,1


In [8]:
# Transformation 4: Created pivot table
# Reason: Created a pivot table to visualize how the air quality index is distributed. This can help out
#.        understand how often poor quality appears. 
pivot_table = api_df.pivot_table(index='air_quality', columns='air_quality', aggfunc=len, fill_value=0)
pivot_table

country                
air_quality       1   2   3   4   5
air_quality                        
1               145   0   0   0   0
2                 0  38   0   0   0
3                 0   0  11   0   0
4                 0   0   0  21   0
5                 0   0   0   0  29

In [9]:
# Calling web dataframe from another notebook
%store -r web_df

In [10]:
# Displaying web dataframe
web_df.head()

,country,2017_vs_1990:_change(%)
3,Afghanistan,348.6
4,Albania,-23.7
5,Algeria,143.5
6,Angola,427.7
7,Anguilla,366.7


In [11]:
# Transformation 5: Filtering columns using inner join
# Reason: Due to mismatch between both dataframe, we will use .merge() function to combine both dataframe
#         which will eliminate unique rows. Since both dataframe do not have data for every country, we
#         we will have to remove them for accuracy purpose.

merge_df = pd.merge(api_df,web_df,on='country')
merge_df

,country,air_quality,2017_vs_1990:_change(%)
0,United Arab Emirates,5,256.3
1,Afghanistan,1,348.6
2,Antigua and Barbuda,2,179.8
3,Anguilla,2,366.7
4,Albania,1,-23.7
...,...,...,...
186,Samoa,1,14.0
187,Yemen,2,81.5
188,South Africa,2,49.7
189,Zambia,1,68.1


In [13]:
%store api_df

Stored 'api_df' (DataFrame)
